# Experiment 003: Lattice-Based Construction with Backward Propagation

This notebook implements:
1. Lattice-based construction for large N values
2. Backward propagation to improve smaller N
3. Proper overlap validation before submission

In [1]:
import copy
import math
import random
import time
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore')

getcontext().prec = 25
scale_factor = Decimal("1e15")

In [2]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal("0.0") * scale_factor, tip_y * scale_factor),
            (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

    def get_params(self):
        return self.center_x, self.center_y, self.angle

    def set_params(self, center_x, center_y, angle):
        self.__init__(str(center_x), str(center_y), str(angle))

    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

In [3]:
def has_collision(trees):
    """Check for collisions between trees"""
    if len(trees) <= 1:
        return False
    for i, tree1 in enumerate(trees):
        for j, tree2 in enumerate(trees):
            if i < j:
                if tree1.polygon.intersects(tree2.polygon) and not tree1.polygon.touches(tree2.polygon):
                    return True
    return False

def calculate_score(trees):
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    score = max(max_x - min_x, max_y - min_y) ** 2 / len(trees)
    return score

def get_side_length(trees):
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    return max(max_x - min_x, max_y - min_y)

In [4]:
# Load the baseline solution (saspav)
def load_submission(filepath):
    df = pd.read_csv(filepath)
    solutions = {}
    for n in range(1, 201):
        group_data = df[df['id'].str.startswith(f"{n:03d}_")]
        trees = []
        for _, row in group_data.iterrows():
            x = str(row['x']).replace('s', '')
            y = str(row['y']).replace('s', '')
            deg = str(row['deg']).replace('s', '')
            trees.append(ChristmasTree(x, y, deg))
        solutions[n] = trees
    return solutions

print("Loading baseline solution...")
baseline = load_submission('/home/code/santa-2025-csv/santa-2025.csv')
print(f"Loaded {len(baseline)} configurations")

# Verify baseline score
baseline_score = sum(calculate_score(baseline[n]) for n in range(1, 201))
print(f"Baseline score: {baseline_score:.6f}")

Loading baseline solution...


Loaded 200 configurations


Baseline score: 70.676102


In [5]:
# Implement backward propagation
def backward_propagation(solutions, verbose=True):
    """Apply backward propagation to improve smaller N configurations."""
    improvements = 0
    
    for n in range(200, 1, -1):
        current_trees = solutions[n]
        target_trees = solutions[n-1]
        target_side = get_side_length(target_trees)
        
        best_side = target_side
        best_tree_to_remove = None
        
        # Try removing each tree
        for tree_idx in range(n):
            candidate = [t.clone() for t in current_trees]
            del candidate[tree_idx]
            
            if not has_collision(candidate):
                candidate_side = get_side_length(candidate)
                if candidate_side < best_side - 1e-10:
                    best_side = candidate_side
                    best_tree_to_remove = tree_idx
        
        if best_tree_to_remove is not None:
            new_trees = [t.clone() for t in current_trees]
            del new_trees[best_tree_to_remove]
            solutions[n-1] = new_trees
            if verbose:
                print(f"n={n-1}: {target_side:.8f} -> {best_side:.8f} (improvement: {target_side - best_side:.8f})")
            improvements += 1
    
    return improvements

print("\nApplying backward propagation...")
solutions = {n: [t.clone() for t in baseline[n]] for n in range(1, 201)}
improvements = backward_propagation(solutions)
print(f"\nTotal improvements: {improvements}")

# Calculate new score
new_score = sum(calculate_score(solutions[n]) for n in range(1, 201))
print(f"\nBaseline score: {baseline_score:.6f}")
print(f"After backward prop: {new_score:.6f}")
print(f"Improvement: {baseline_score - new_score:.6f}")


Applying backward propagation...



Total improvements: 0



Baseline score: 70.676102
After backward prop: 70.676102
Improvement: 0.000000


In [6]:
# Validate no overlaps in all configurations
print("\nValidating all configurations for overlaps...")
all_valid = True
for n in range(1, 201):
    if has_collision(solutions[n]):
        print(f"  n={n}: OVERLAP DETECTED!")
        all_valid = False

if all_valid:
    print("All configurations are valid (no overlaps)!")
else:
    print("Some configurations have overlaps - reverting to baseline for those!")


Validating all configurations for overlaps...
  n=30: OVERLAP DETECTED!
  n=41: OVERLAP DETECTED!
  n=43: OVERLAP DETECTED!
  n=61: OVERLAP DETECTED!


  n=77: OVERLAP DETECTED!


  n=110: OVERLAP DETECTED!
  n=114: OVERLAP DETECTED!


  n=117: OVERLAP DETECTED!


  n=134: OVERLAP DETECTED!
  n=137: OVERLAP DETECTED!


  n=170: OVERLAP DETECTED!
  n=171: OVERLAP DETECTED!


Some configurations have overlaps - reverting to baseline for those!


In [7]:
# Create submission
def create_submission(solutions, filepath):
    rows = []
    for n in range(1, 201):
        for i, tree in enumerate(solutions[n]):
            rows.append({
                'id': f"{n:03d}_{i}",
                'x': f"s{float(tree.center_x)}",
                'y': f"s{float(tree.center_y)}",
                'deg': f"s{float(tree.angle)}"
            })
    df = pd.DataFrame(rows)
    df.to_csv(filepath, index=False)
    return df

# Save submission
submission = create_submission(solutions, '/home/submission/submission.csv')
submission.to_csv('/home/code/experiments/003_lattice/submission.csv', index=False)
print(f"Saved submission with {len(submission)} rows")

# Final score
final_score = sum(calculate_score(solutions[n]) for n in range(1, 201))
print(f"\nFinal score: {final_score:.6f}")
print(f"Target: 68.922808")
print(f"Gap to target: {final_score - 68.922808:.6f}")

Saved submission with 20100 rows



Final score: 70.676102
Target: 68.922808
Gap to target: 1.753294


In [8]:
# Save metrics
import json
metrics = {
    'baseline_score': baseline_score,
    'final_score': final_score,
    'improvement': baseline_score - final_score,
    'backward_prop_improvements': improvements,
    'target': 68.922808
}
with open('/home/code/experiments/003_lattice/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)
print("Metrics saved")

Metrics saved


In [9]:
# Investigate the overlap detection - check if it's a precision issue
# The saspav solution should be valid - let's check the intersection areas

def check_overlap_details(trees, n):
    """Check overlap details for a configuration"""
    overlaps = []
    for i, tree1 in enumerate(trees):
        for j, tree2 in enumerate(trees):
            if i < j:
                if tree1.polygon.intersects(tree2.polygon) and not tree1.polygon.touches(tree2.polygon):
                    intersection = tree1.polygon.intersection(tree2.polygon)
                    overlaps.append((i, j, intersection.area / 1e30))  # Scale down from 1e15^2
    return overlaps

# Check one of the flagged configurations
for n in [30, 41, 77]:
    overlaps = check_overlap_details(solutions[n], n)
    print(f"n={n}: {len(overlaps)} overlaps")
    for i, j, area in overlaps[:3]:
        print(f"  Trees {i} and {j}: intersection area = {area:.2e}")

n=30: 1 overlaps
  Trees 16 and 22: intersection area = 0.00e+00
n=41: 1 overlaps
  Trees 3 and 21: intersection area = 1.56e-32
n=77: 2 overlaps
  Trees 11 and 29: intersection area = 0.00e+00
  Trees 48 and 72: intersection area = 0.00e+00


## Lattice-Based Construction

Now let's try the egortrushin approach: create a 2-tree unit cell and tile it to generate large N configurations.

In [11]:
# Lattice-based construction from egortrushin kernel
# Key idea: Create 2-tree unit cell, tile with fractional translations

def translate_trees(base_trees, lengthx, lengthy, nt):
    """Tile base trees using translation pattern"""
    trees = []
    for tree in base_trees:
        for x in range(nt[0]):
            for y in range(nt[1]):
                new_tree = ChristmasTree(
                    center_x=tree.center_x + Decimal(str(x * lengthx)),
                    center_y=tree.center_y + Decimal(str(y * lengthy)),
                    angle=tree.angle
                )
                trees.append(new_tree)
    return trees

# Create initial 2-tree unit cell (from egortrushin kernel)
initial_trees = [
    ChristmasTree(-2.93069232, -4.24856960, 67),
    ChristmasTree(-3.92971914, -4.16631769, 250.00)
]

print("Initial 2-tree unit cell:")
for i, t in enumerate(initial_trees):
    print(f"  Tree {i}: x={float(t.center_x):.4f}, y={float(t.center_y):.4f}, angle={float(t.angle):.1f}")

# Test lattice generation for N=72 (4x9 grid of 2-tree cells)
nt = [4, 9]  # 4 columns, 9 rows = 72 trees
lengthx = 1.0  # Initial translation
lengthy = 0.9

lattice_trees = translate_trees(initial_trees, lengthx, lengthy, nt)
print(f"\nGenerated {len(lattice_trees)} trees for N=72")
print(f"Has collision: {has_collision(lattice_trees)}")
if not has_collision(lattice_trees):
    score = calculate_score(lattice_trees)
    print(f"Score: {score:.6f}")
    print(f"Baseline N=72 score: {calculate_score(baseline[72]):.6f}")

Initial 2-tree unit cell:
  Tree 0: x=-2.9307, y=-4.2486, angle=67.0
  Tree 1: x=-3.9297, y=-4.1663, angle=250.0

Generated 72 trees for N=72
Has collision: True
